# # **House Rocket - Comprando imóveis**
por [Lucas Capovilla Cassimiro](http://www.linkedin.com/in/lucascapovilla)

A House Rocket é uma plataforma digital que tem como modelo de negócio, a compra e a venda de imóveis usando tecnologia. Sua principal estratégia é comprar boas casas em ótimas localizações com preços baixos e depois revendê-las posteriormente à preços mais altos. Quanto maior a diferença entre a compra e a venda, maior o lucro da empresa e portanto maior sua receita.

Entretanto, as casas possuem muitos atributos que as tornam mais ou menos atrativas aos compradores e vendedores e a localização e o período do ano também podem influenciar os preços. A seguir vamos responder as seguintes perguntas:

1. Quais recursos de uma casa mais impactam no preço?
2. A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças?
3. Uma vez a casa em posse da empresa, qual o melhor momento para vendê-la?

In [ ]:
# Análise dos dados e data wrangling
import pandas as pd
import numpy as np
import random as rnd

# Visualização dos dados
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
housesalesdata = pd.read_csv("../input/housesalesprediction/kc_house_data.csv") #carregando o banco de dados
housesalesdata

# ***1. Quais recursos de uma casa mais impactam no preço?***

Utilizando a Correlação de Pearson, podemos entender como as variáveis se relacionam e mais precisamente, podemos analisar quais são as que mais incluenciam no valor de uma casa. Para encontrar essa resposta é necessário verificar o valor de correlação apresentado abaixo e identificar quais são relevantes. Podemos dividir as correlações em 5 níveis diferentes:

* **0.00 a 0.19:** Correlação muito fraca
* **0.20 a 0.39:** Correlação fraca
* **0.40 a 0.69:** Correlação moderada
* **0.70 a 0.89:** Correlação forte
* **0.90 a 1.00:** Correlação muito forte

Observando o heatmap abaixo podemos concluir que a quantidade de banehiros(bathrooms), a metragem quadrada dos 15 vizinhos mais próximos(sqft_living15), a metragem quadrada dos andares acima do térreo(sqft_above), a nota em relação à construção/design (grade) e metragem quadrada interna da casa (sqft_living) possuem os maiores graus de correlação com o preço das casas.

In [ ]:
sns.set(style="whitegrid", font_scale=1)

plt.figure(figsize=(15,15))
plt.title('Correlação de Pearson',fontsize=25)
sns.heatmap(housesalesdata.corr(),linewidths=0.25,vmax=0.7,square=True,cmap="GnBu",linecolor='w',
            annot=True, annot_kws={"size":7}, cbar_kws={"shrink": .7})


Podemos ver abaixo em ordem decrescente o grau de correlação do preço das casas em relação às demais variáveis.

In [ ]:
price_corr = housesalesdata.corr()['price'].sort_values(ascending=False)
print(price_corr)

# ***2. A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças?***

O objetivo reformar um imóvel para depois vendê-lo não é outro se não valorizar este bem. Portanto, vamos olhar com mais atenção aos itens que mais incluenciam no preço de um imóvel. A começar pelo número de banheiros, podemos observar no boxplot abaixo que o valor dos imóveis tendem a aumentar ligeiramente conforme o número de banheiros disponíveis, portanto, um bom investimento seria a construção de mais banheiros no terreno.

In [ ]:
f, axe = plt.subplots(1, 1,figsize=(8,5))
sns.boxplot(x=housesalesdata['bedrooms'],y=housesalesdata['price'])
axe.set(xlabel='Banheiros', ylabel='Preço', title='N° de banheiros vs Preço')

A metragem quadrada dos 15 vizinhos mais próximos está entre os itens que mais influenciam o preço de um imóvel, porém não é algo que possa ser alterado. Vamos então comparar a metragem quadrada construída acima do térreo e a metragem quadrada total em relação ao preço. É evidente, através dos gráficos a seguir que quanto maior as áreas construídas, o valor dos imóveis tende a aumentar.


In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.scatterplot(x=housesalesdata['sqft_above'],y=housesalesdata['price'], ax=axes[0])
sns.scatterplot(x=housesalesdata['sqft_living'],y=housesalesdata['price'], ax=axes[1])
sns.despine(bottom=True, left=True)
axes[0].set(xlabel='m² de construção acima do térreo', ylabel='Preço em milhões [USD]', title='Preço vs Área de construção acima do térreo')
axes[1].set(xlabel='m² de construção interna da casa', ylabel='Preço em milhões [USD]', title='Preço vs Área de construção total')
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()

Por ultimo, temos a nota do imóvel, que é dada em relação à construção e design. Se observarmos a comparação da nota do imóvel em relação ao preço, fica evidente que quanto maior a nota do imóvel, maior o valor deste.


In [ ]:
f, axe = plt.subplots(1, 1,figsize=(15,5))
sns.boxplot(x=housesalesdata['grade'],y=housesalesdata['price'], ax=axe)
sns.despine(left=True, bottom=True)
axe.set(xlabel='Nota', ylabel='Preço', title='Comparação entre nota e preço')

A nota do imóvel tem uma correlação moderada(quase forte) com o preço dos imóveis e portanto é interessante buscar aumentar esta nota e consequentemente o imóvel será valorizado. Neste momento surge a pergunta: mas como?

A resposta é bem simples: entendendo o grau de correlação da nota com as demais variáveis, observando quais apresentam uma correlação moderadamente forte, forte ou muito forte. Dessa forma vamos encontrar os seguintes itens:

1. Área de construção interna (m²)
2. Área de construção acima do térreo (m²)
3. Número de banheiros

Os itens 1 e 2 têm uma forte correlação com a nota, o que significa que seria interessante aumentar a metragem interna, seja construindo um novo cômodo ou ampliando algum existente. O número de banheiros possui uma correlação moderada com a nota, não impacta tanto quanto as duas anteriores, porém aumentar o número deles sem dúvidas influenciará no valor de venda.

In [ ]:
price_corr = housesalesdata.corr()['grade'].sort_values(ascending=False)
print(price_corr)

# 3. Uma vez a casa em posse da empresa, qual o melhor momento para vendê-la e qual seria o preço da venda?

Para saber o melhor momento de se vender um imóvel, precisamos observar como os preços se comportam ao longo do tempo. Para conseguir observar com mais clareza, primeiro será necessário converter a coluna de data para o formato correto e criar duas novas colunas sendo elas mês e ano.

In [ ]:
housesalesdata['date'] = pd.to_datetime(housesalesdata['date']) #convertendo para tipo de dado correto

housesalesdata['month'] = housesalesdata['date'].apply(lambda date:date.month) #criando uma nova coluna de mes
housesalesdata['year'] = housesalesdata['date'].apply(lambda date:date.year) #criando uma nova coluna de ano

print(housesalesdata.columns.values) #verificando se as colunas foram adicionadas corretamente

Em seguida vamos gerar um gráfico que permita a visualização dos preços dos imóveis ao longo dos meses. Pode-se concluir a partir das informações disponíveis abaixo que há um pico nos preços em abril e esse aumento geral se mantém um pouco até junho. Podemos concluir então que o m~es de abril é o ideal para a venda de imóveis, seguido de maio e junho onde os valores ainda se mantém acima em relação aos demais meses.

In [ ]:
f, axe = plt.subplots(1, 1,figsize=(15,5))
housesalesdata.groupby('month').mean()['price'].plot()
sns.despine(left=True, bottom=True)
axe.set(xlabel='Mês', ylabel='Preço', title='Tendência dos preços ao longo do ano')

In [ ]:
housesalesdata.head() #visualisando dados (5 primeiras linhas)

In [ ]:
housesalesdata.tail() #visualisando dados (5 ultimas linhas)

In [ ]:
housesalesdata.columns.values #retorna o nome das colunas em um array

In [ ]:
housesalesdata.isnull().sum() #verifica quantos espaços vazios há em cada coluna

In [ ]:
housesalesdata.info() #verifica o tipo de dado que está em cada coluna

In [ ]:
housesalesdata.describe().transpose() #Entendendo a distribuição numérica do banco de dados

# Referências

Contextualização do case: [Clique aqui](https://medium.com/@meigarom/os-5-projetos-de-data-science-que-far%C3%A1-o-recrutador-olhar-para-voc%C3%AA-c32c67c17cc9)

Boxplot em python: . [Clique aqui](https://pythonbasics.org/seaborn-boxplot/)

Data Cleaning: [Clique aqui](https://miamioh.instructure.com/courses/38817/pages/data-cleaning)

